# Assignment 1 - Rhythm Generator
### Mariella Daghfal, Kaede Johnson, Shayan Khajehnouri, & Tahel Singer

In [114]:
# to be safe i personally created venv with anaconda terminal.
# conda create -n "digital_musicology" python=3.8
# conda activate digital_musicology
# conda install pip

In [115]:
## Dependencies
# pip install music21
# pip install tqdm
# pip install pandas
from music21 import *
from tqdm import *
import pandas as pd
import numpy as np
import collections

## for viewing score, download and install finale note pad at https://store.makemusic.com/Downloads/Default.aspx?id=538&_ga=2.3616692.58482559.1678537005-1850486829.1678537005 and then run:
# configure.run()

## I don't know if it's because I screwed up something during initial setup but this is what I had to do to get music showing:
# environment.set('musescoreDirectPNGPath', 'C:\\Program Files (x86)\\Finale NotePad 2012\\Finale NotePad.exe')

In [116]:
## Bach score from package

s = corpus.parse('bach/bwv65.2.xml')

# Print music12 object to show it worked
# s.show('text')

# image version (ideally this is inline but i can't get that working; for me it pulls open finale note pad with the sheet music in a separate window)
# s.show()

In [117]:
## Load scores into corpus
# Be sure to modify folder path below
# Excludes scores that aren't strictly 3/4 meter all the way through (only 15 / 600)

import os
import tqdm
slangpolska_scores = {}
excluded = 0
# CHANGE FOLDER BELOW TO LOCATION OF XML FILES. Download xml folders from https://github.com/DCMLab/music-generation-challenge-2021/tree/main/data
xml_file_location = 'C:/Users/HP/anaconda3/envs/digital_musicology/xml/'
scores_xml = os.listdir(xml_file_location)
for si in trange(len(scores_xml)):
    if scores_xml[si] != 'conversion.log':
        tmp = converter.parse(xml_file_location + scores_xml[si])
        tmp_meters = tmp.recurse().getElementsByClass(meter.TimeSignature)
        if len(tmp_meters) != 1 or tmp_meters[0].ratioString != '3/4':
            print('excluding ' + scores_xml[si] + '. ' + str(len(tmp_meters)) + ' meter(s), first is ' + tmp_meters[0].ratioString)
            excluded += 1
        else:
            slangpolska_scores[scores_xml[si]] = tmp

print('Imported ' + str(len(slangpolska_scores)) + ' scores into corpus. Excluded ' + str(excluded) + ' due to inconsistent meter.')


  6%|▌         | 37/600 [00:01<00:23, 24.47it/s]

excluding Ninas_slängpolska_20240b.xml. 2 meter(s), first is 3/4


 24%|██▍       | 146/600 [00:03<00:04, 94.87it/s]

excluding Poloness_ur_Andreas_Hööks_notbok_cae293.xml. 4 meter(s), first is 2/2
excluding Polonoise_92b598.xml. 2 meter(s), first is 3/4


 49%|████▉     | 293/600 [00:04<00:03, 88.04it/s] 

...

excluding _Polonäs_sexdregasamlingen_del_1_nr_36_dfc2c9.xml. 2 meter(s), first is 3/4
excluding _Polonäs_sexdregasamlingen_del_1_nr_36_ed2b21.xml. 2 meter(s), first is 3/4


 95%|█████████▍| 569/600 [00:09<00:00, 79.50it/s]

excluding _Polska_efter_Gustav_Strutz_21bb05.xml. 5 meter(s), first is 3/4
excluding _Polska_efter_Gustav_Strutz_5b7324.xml. 5 meter(s), first is 3/4


100%|██████████| 600/600 [00:10<00:00, 59.16it/s]

excluding _Polska_Kringellåt_efter_Snickar_Erik_985b63.xml. 3 meter(s), first is 4/4
Imported 584 scores into corpus. Excluded 15 due to inconsistent meter.


In [36]:
# Investigate one score to get familiar with music21
sample_score = slangpolska_scores['_1814_698087.xml']
print('music21 object for imported score is called ' + str(type(sample_score)))
#print('sample score key is ' + str(sample_score.analyze('key')))
print('The first time signature object is ' + sample_score.recurse().getElementsByClass(meter.TimeSignature)[0].ratioString)
print('There is (are) ' + str(len(sample_score.recurse().getElementsByClass(meter.TimeSignature))) + ' time signature object(s)')
print('Measure 6 does not have a time signature object: ' + str(sample_score.measure(6).timeSignature is None))
print('Recall the sample score is in time ' + sample_score.recurse().getElementsByClass(meter.TimeSignature)[0].ratioString)
curr_measure = 0
for i in sample_score.recurse().notes:
    if i.measureNumber != curr_measure:
        curr_measure = i.measureNumber
        print("Measure " + str(curr_measure))
    print('Beat: ' + str(i.beat) + ', Duration: ' + str(i.duration.quarterLength) + ', Pitch: ' + str(i.name))
print('Music21 object print:')
sample_score.show('text')

music21 object for imported score is called <class 'music21.stream.base.Score'>
The first time signature object is 3/4
There is (are) 1 time signature object(s)
Measure 6 does not have a time signature object: True
Recall the sample score is in time 3/4
Measure 1
Beat: 1.0, Duration: 0.5, Pitch: D
Beat: 1.5, Duration: 0.5, Pitch: C
Beat: 2.0, Duration: 1.0, Pitch: B
Beat: 3.0, Duration: 1.0, Pitch: G
Measure 2
Beat: 1.0, Duration: 0.5, Pitch: C
Beat: 1.5, Duration: 0.5, Pitch: B
Beat: 2.0, Duration: 1.0, Pitch: A
Beat: 3.0, Duration: 1.0, Pitch: D
Measure 3
Beat: 1.0, Duration: 0.75, Pitch: G
Beat: 1.75, Duration: 0.25, Pitch: F#
Beat: 2.0, Duration: 0.5, Pitch: G
Beat: 2.5, Duration: 0.5, Pitch: A
Beat: 3.0, Duration: 0.5, Pitch: B
Beat: 3.5, Duration: 0.5, Pitch: C
Measure 4
Beat: 1.0, Duration: 0.5, Pitch: D
Beat: 1.5, Duration: 0.25, Pitch: D
Beat: 1.75, Duration: 0.25, Pitch: E
Beat: 2.0, Duration: 0.5, Pitch: D
Beat: 2.5, Duration: 0.5, Pitch: C
Beat: 3.0, Duration: 0.5, Pitch: B

In [106]:
# Tokenization
# Starting simple until we define rhythm. Tokens will simply be location of beat in measure (that is, first element in Beat: , Duration: , Pitch: printout sequence above)
# First, observe where beats can occur (also sneakily multiply beat timings by 12 because see output + explanation in next cell)
import collections
all_beats = []
tokenized_corpus = {}
for score in slangpolska_scores:
    timing_tokens = []
    for note in slangpolska_scores[score].recurse().notes:
        all_beats.append(note.beat)
        timing_tokens.append(float(note.beat * 12))
    tokenized_corpus[score] = timing_tokens
collections.Counter(all_beats).most_common(50)

[(1.0, 9486),
 (2.0, 9320),
 (3.0, 8826),
 (1.5, 7895),
 (2.5, 7073),
 (1.75, 6077),
 (3.5, 5808),
 (2.75, 4338),
 (1.25, 3120),
 (3.75, 2736),
 (2.25, 2520),
 (3.25, 1692),
 (Fraction(5, 3), 33),
 (Fraction(4, 3), 29),
 (Fraction(8, 3), 22),
 (Fraction(7, 3), 21),
 (Fraction(10, 3), 19),
 (Fraction(11, 3), 18),
 (Fraction(11, 6), 10),
 (1.875, 10),
 (2.875, 8),
 (3.875, 6),
 (2.125, 6),
 (1.375, 5),
 (Fraction(7, 6), 5),
 (Fraction(19, 6), 3),
 (Fraction(17, 6), 3),
 (1.625, 3),
 (2.625, 3),
 (2.375, 2),
 (Fraction(13, 6), 2),
 (Fraction(23, 12), 2),
 (Fraction(23, 6), 2),
 (3.625, 2),
 (nan, 1),
 (nan, 1)]

In [113]:
# Beat b can occur along the interval 1 <= b < 4. Note some beats occur on a fraction with maximum denominator 12.
# Therefore, to deal with one data type (floats) our tokens are beat timings multiplied by twelve. We can theoretically divide our generated beat timings by 12 to map back to reality.
# That said, this script is just working toward a proof of concept right now, and I suspect our tokenziation will change. Also I bet floats are awful for NLP models so if we end up doing
# an RNN or working with tensors we'll probably do what xml does - integers based on note size. I just wanted to move forward on a workflow.
# TODO: Linearize repeat bars
# TODO: Decide what rhythm is and create smarter tokens
# TODO: Remove or handle the 1-2 score(s) with nan beat timings
from itertools import chain
collections.Counter(list(chain(*tokenized_corpus.values()))).most_common(50)


[(12.0, 9486),
 (24.0, 9320),
 (36.0, 8826),
 (18.0, 7895),
 (30.0, 7073),
 (21.0, 6077),
 (42.0, 5808),
 (33.0, 4338),
 (15.0, 3120),
 (45.0, 2736),
 (27.0, 2520),
 (39.0, 1692),
 (20.0, 33),
 (16.0, 29),
 (32.0, 22),
 (28.0, 21),
 (40.0, 19),
 (44.0, 18),
 (22.0, 10),
 (22.5, 10),
 (34.5, 8),
 (46.5, 6),
 (25.5, 6),
 (16.5, 5),
 (14.0, 5),
 (38.0, 3),
 (34.0, 3),
 (19.5, 3),
 (31.5, 3),
 (28.5, 2),
 (26.0, 2),
 (23.0, 2),
 (46.0, 2),
 (43.5, 2),
 (nan, 1),
 (nan, 1)]

In [110]:
# TODO: Random rhythm gen, smart rhythm gen, discover rhythm visualizer and comparison metric...

foo
